# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [80]:
# Dependencies and Setup
import pandas as pd
import hvplot.pandas
import requests
# Import API key
from api_keys import geoapify_key

In [86]:
# Load the CSV file created in Part 1 into a Pandas DataFrame, clean data.
city_data_df = pd.read_csv("../output_data/cities_ferrier.csv")
# Clean and display
clean_df = city_data_df.dropna(how = "any").reset_index()
clean_df.head()

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,0,lagunas,-5.2269,-75.6753,93.81,38.0,35.0,1.90,PE,1.717354e+09
1,1,1,mount isa,-20.7333,139.5000,40.77,93.0,83.0,4.61,AU,1.717354e+09
2,2,2,ribeira grande,38.5167,-28.7000,66.60,88.0,75.0,10.36,PT,1.717354e+09
3,3,3,port-aux-francais,-49.3500,70.2167,33.01,69.0,97.0,42.19,TF,1.717354e+09
4,5,5,wenchang,30.9040,118.4887,67.26,67.0,100.0,3.58,CN,1.717354e+09


In [102]:
# checking info
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567 entries, 0 to 566
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       567 non-null    int64  
 1   City_ID     567 non-null    int64  
 2   City        567 non-null    object 
 3   Lat         567 non-null    float64
 4   Lng         567 non-null    float64
 5   Max Temp    567 non-null    float64
 6   Humidity    567 non-null    float64
 7   Cloudiness  567 non-null    float64
 8   Wind Speed  567 non-null    float64
 9   Country     567 non-null    object 
 10  Date        567 non-null    float64
dtypes: float64(7), int64(2), object(2)
memory usage: 48.9+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [87]:
# Create heatmap of cities
map_heatmap = clean_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    color='Humidity',
    cmap='inferno'
)

map_heatmap

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [112]:
# Narrow down cities that fit criteria and drop any results with null values
# Null values already dropped in clean_df
# Weather conditions modified to have at least 5 cities, example conditions had 0
filtered_df = clean_df[(clean_df['Max Temp'] < 28) &  
                       (clean_df['Cloudiness'] != 0) & 
                       (clean_df['Humidity'] > 70)]
filtered_df

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,21,21,grytviken,-54.2811,-36.5092,26.28,72.0,99.0,4.41,GS,1.717354e+09
27,28,28,tazovsky,67.4667,78.7000,17.10,96.0,25.0,7.56,RU,1.717354e+09
31,32,32,dudinka,69.4058,86.1778,23.68,93.0,100.0,3.00,RU,1.717354e+09
132,136,136,vorgashor,67.5833,63.9500,17.80,96.0,66.0,4.70,RU,1.717354e+09
383,406,406,talnakh,69.4865,88.3972,23.20,97.0,99.0,5.28,RU,1.717354e+09
458,487,487,igarka,67.4667,86.5833,27.01,90.0,81.0,5.99,RU,1.717354e+09


In [114]:
# Proof that suggested weather returned 0 examples 
proof_filtered_df = clean_df[(clean_df['Max Temp'] < 27) & (clean_df['Max Temp'] > 21) & 
                       (clean_df['Wind Speed'] < 4.5) & 
                       (clean_df['Cloudiness'] == 0)]
proof_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       0 non-null      int64  
 1   City_ID     0 non-null      int64  
 2   City        0 non-null      object 
 3   Lat         0 non-null      float64
 4   Lng         0 non-null      float64
 5   Max Temp    0 non-null      float64
 6   Humidity    0 non-null      float64
 7   Cloudiness  0 non-null      float64
 8   Wind Speed  0 non-null      float64
 9   Country     0 non-null      object 
 10  Date        0 non-null      float64
dtypes: float64(7), int64(2), object(2)
memory usage: 0.0+ bytes


### Step 3: Create a new DataFrame called `hotel_df`.

In [117]:
# Create hotel_df from ideal_df with selected columns and an empty 'Hotel Name' column
hotel_df = filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df['Hotel Name'] = ''  # Add an empty column for 'Hotel Name'

# Display the hotel_df DataFrame
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
20,grytviken,GS,-54.2811,-36.5092,72.0,
27,tazovsky,RU,67.4667,78.7000,96.0,
31,dudinka,RU,69.4058,86.1778,93.0,
132,vorgashor,RU,67.5833,63.9500,96.0,
383,talnakh,RU,69.4865,88.3972,97.0,
458,igarka,RU,67.4667,86.5833,90.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [138]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
hotel_df

Starting hotel search
grytviken - nearest hotel: No hotel found
tazovsky - nearest hotel: Тазовчанка
dudinka - nearest hotel: Виктория
vorgashor - nearest hotel: София
talnakh - nearest hotel: Талнах
igarka - nearest hotel: Заполярье


,City,Country,Lat,Lng,Humidity,Hotel Name
20,grytviken,GS,-54.2811,-36.5092,72.0,No hotel found
27,tazovsky,RU,67.4667,78.7000,96.0,Тазовчанка
31,dudinka,RU,69.4058,86.1778,93.0,Виктория
132,vorgashor,RU,67.5833,63.9500,96.0,София
383,talnakh,RU,69.4865,88.3972,97.0,Талнах
458,igarka,RU,67.4667,86.5833,90.0,Заполярье


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [141]:
%%capture --no-display
map_heatmap = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    color='Humidity',
    cmap='viridis',
    hover_cols=['City', 'Country', 'Hotel Name']
)

map_heatmap

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)